<h1><center>Laboratorio 5: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: SIVERT ESCAFF 
- Nombre de alumno 2: RODRIGO MONTECINO


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/...../)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Jueves a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [2]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [3]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [4]:
import pickle

# route
file_path = 'C:/online_retail_data.pickle'

# open
with open(file_path, 'rb') as file:
    df_retail = pickle.load(file)

# print head para entender el archivo
df_retail.sample(5)







,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
291022,ID525995,21929,JUMBO BAG PINK VINTAGE PAISLEY,1.0,2010-10-08 11:31:00,1.95,17196.0,United Kingdom
256486,ID522415,90040D,MURANO STYLE GLASS BRACELET PINK,1.0,2010-09-14 12:06:00,3.50,16746.0,United Kingdom
325254,ID529799,22639,SET OF 4 NAPKIN CHARMS HEARTS,6.0,2010-10-31 10:03:00,2.55,16756.0,United Kingdom
228932,ID518950,22070,SMALL RED RETROSPOT MUG IN BOX,6.0,2010-08-12 15:06:00,3.75,15296.0,United Kingdom
41173,ID494922,22090,PAPER BUNTING RETRO SPOTS,6.0,2010-01-20 09:57:00,2.95,12987.0,United Kingdom


In [5]:
df_retail.shape

(400947, 8)

### 1. Función para explorar características [5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [3 puntos]
- Imprima un conteo de datos nulos por variable [2 puntos]

NOTA: Para generar los gráficos es **OBLIGATORIO** el uso de plotly.

**Respuesta:**

In [ ]:

def explore_data(dataframe_in):
    # Histograma para la variable 'Price'
    fig_price = go.Figure()
    fig_price.add_trace(go.Histogram(x=dataframe_in['Price'], name='Price'))
    fig_price.update_layout(title_text='Histogram of Prices', xaxis_title='Price', yaxis_title='Count')
    fig_price.show()

    # Histograma para la variable 'Quantity'
    fig_quantity = go.Figure()
    fig_quantity.add_trace(go.Histogram(x=dataframe_in['Quantity'], name='Quantity'))
    fig_quantity.update_layout(title_text='Histogram of Quantity', xaxis_title='Quantity', yaxis_title='Count')
    fig_quantity.show()

    # Conteo de datos nulos por variable
    missing_values = dataframe_in.isnull().sum()
    print("Conteo de datos nulos por variable:")
    print(missing_values)

explore_data(df_retail)

### 2. Eliminando outliers [10 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

Hint: tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).
>**Nota: No modificar el método set_output de la clase IQR**

**Respuesta:**

In [26]:
class IQR(BaseEstimator, TransformerMixin):
    def __init__(self, factor=1.5):
        # 'factor' es el multiplicador del IQR 
        self.factor = factor

    def fit(self, X, y=None):
        # calcular el primer y tercer cuartil y el rango intercuartilico 
        self.Q1 = X.quantile(0.25)
        self.Q3 = X.quantile(0.75)
        self.IQR = self.Q3 - self.Q1

        # identificar outliers
        self.lower_bound = self.Q1 - self.factor * self.IQR
        self.upper_bound = self.Q3 + self.factor * self.IQR

        return self

    def transform(self, X):
        # Filtrar outliers y devolver solo los valores que estan dentro de los limites
        return X[(X >= self.lower_bound) & (X <= self.upper_bound)]

    def set_output(self, transform='default'):
        return self

#### 2.2 Creación del Pipeline [5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [1 punto]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [1 punto]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [1 punto]


HINT: El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.
>**Nota: Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.**

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
# definimos las columnas numericas y categoricas
categorical_columns = df_retail.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_columns = df_retail.select_dtypes(include=['int64', 'float64']).columns.tolist()

# definir el pipeline de transformaciones numéricas ya que el transformador creado anteriormente es solo util para variables numericas 
numeric_transformations = Pipeline(steps=[
    ('iqr', IQR(factor=1.5))  
])

# Crear el ColumnTransformer
column_transformer = ColumnTransformer(transformers=[
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', 'passthrough', categorical_columns)
], verbose_feature_names_out=False)

# ajustamos el metodo set_output del transformer para usar salida en formato DataFrame
column_transformer.set_output(transform='pandas')


# apicamos el ColumnTransformer a los datos
df_iqr = column_transformer.fit_transform(df_retail)


# mostramos el antes y despues con la funcion explore definida anteriormente 
explore_data(df_retail)
explore_data(df_iqr)

El factor que es elige es 1.5 ya que en la bibliografia buscada es el mas comun. Notamos que como esta definido la estructura IQR, sus limites se definen como  lower_bound = Q1 - factor * IQR  y upper_bound = Q3 + factor * IQR   . Vemos que si el factor aumenta el limite inferior  disminuye de valor y sucede lo mismo con el limite superior el cual aumenta si el factor aumenta , esto tiene como consecuencia que se filtran menos valores ya que el intervalo (lower_bound, upper_bound) es mas grande .

*Reporte los cambios observados aquí.*

### 3. Agregando un imputer al pipeline [10 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [1 punto]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [1 punto]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [2 puntos]

>**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [36]:
from sklearn.impute import SimpleImputer



# Pipeline para variables categóricas que imputa usando el valor más frecuente
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

# Pipeline para var numerica
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

# Crear el ColumnTransformer
column_transformer = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_columns),
        ('num', numeric_transformer, numerical_columns)
    ],
    verbose_feature_names_out=False
)

# configuuramos para obtener un DataFrame como salida
column_transformer.set_output(transform='pandas')
df_mean_imputer = column_transformer.fit_transform(df_retail)

# imprimois 
df_mean_imputer.head()




          Country                          Description  Quantity  Price
0  United Kingdom  15CM CHRISTMAS GLASS BALL 20 LIGHTS      12.0   6.95
1  United Kingdom                   PINK CHERRY LIGHTS      12.0   6.75
2  United Kingdom                  WHITE CHERRY LIGHTS      12.0   6.75
3  United Kingdom         RECORD FRAME 7" SINGLE SIZE       48.0   2.10
4  United Kingdom       STRAWBERRY CERAMIC TRINKET BOX      24.0   1.25


In [ ]:
explore_data(df_mean_imputer)

4. Notamos que df_mean_imputer no contiene nulos en ninguna columna, pero los outliers siguen vivos  esto hace que no se pueda interpretar los datos de ninguna manera

In [42]:
from sklearn.impute import KNNImputer

# Modificar el pipeline para variables numericas usando KNNImputer
numeric_transformer_knn = Pipeline([
    ('imputer', KNNImputer(n_neighbors=1))  
])

# Crear el ColumnTransformer incluyendo el nuevo pipeline numerico
column_transformer_knn = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_columns),
        ('num', numeric_transformer_knn, numerical_columns)
    ],
    verbose_feature_names_out=False
)

# Configurar para obtener un DataFrame como salida
column_transformer_knn.set_output(transform='pandas')


# Aplicar el ColumnTransformer al DataFrame original
df_knn_imputer = column_transformer_knn.fit_transform(df_retail)

df_knn_imputer.sample(5)



          Country                          Description  Quantity  Price
0  United Kingdom  15CM CHRISTMAS GLASS BALL 20 LIGHTS      12.0   6.95
1  United Kingdom                   PINK CHERRY LIGHTS      12.0   6.75
2  United Kingdom                  WHITE CHERRY LIGHTS      12.0   6.75
3  United Kingdom         RECORD FRAME 7" SINGLE SIZE       48.0   2.10
4  United Kingdom       STRAWBERRY CERAMIC TRINKET BOX      24.0   1.25


In [ ]:
explore_data(df_knn_imputer)

6. se puede observar que el cambio aplicando este pipe no es muy significativo de hecho no cambio casi nada 
7. la mejor opcion hasta el moomento es usar el df_iqr donde se eliminar oulinear y se queda los miissing values 

### 4. Creación de nuevas features [20 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [10 puntos]
(2 puntos por cada custom feature)

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [44]:
import pandas as pd

def custom_features(dataframe_in):
    df = dataframe_in.copy()
    
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    now = df['InvoiceDate'].max() + pd.Timedelta(days=1)

    # preparamos el dataframe
    aggregation_functions = {
        'InvoiceDate': ['max', 'min', lambda x: (now - x.max()).days, 'count', lambda x: x.diff().dt.days.std(ddof=0)],
        'Total': 'sum'
    }
    df_grouped = df.groupby('Customer ID').agg(aggregation_functions)

    # Renombrar columnas y calcular valores finales para cada LRMFP
    df_grouped.columns = ['LastPurchaseDate', 'FirstPurchaseDate', 'Recency', 'Frequency', 'Periodicity', 'Monetary']
    df_grouped['Length'] = (df_grouped['LastPurchaseDate'] - df_grouped['FirstPurchaseDate']).dt.days
    df_grouped['Periodicity'].fillna(0, inplace=True)  # reemplaamos NaN por 0 en periodicidad para clientes con una sola compra
    
    # selecionamos y ordenamos los index
    df_features = df_grouped[['Length', 'Recency', 'Frequency', 'Monetary', 'Periodicity']].reset_index()
    
    return df_features


#### 4.2 Agregando las custom features [10 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [1 punto]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [1 punto]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [45]:

# creamos ar el FunctionTransformer envolviendo la función custom_features
custom_features_transformer = FunctionTransformer(custom_features)

# sea crea el ColumnTransformer
col_transformer = ColumnTransformer(transformers=[], remainder='passthrough')

# pipeline completo
retail_pipeline = Pipeline(steps=[
    ('col_transformer', col_transformer),   # Primer paso del pipeline
    ('custom_features', custom_features_transformer)  # Segundo paso del pipeline
])


# apliocamos el piipeline 
df_custom = retail_pipeline.fit_transform(df_retail)

explore_data(df_custom)

#epicfail

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

### 5. MinMax Scaler [10 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [46]:
class MinMax(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        self.min_ = X.min()
        self.max_ = X.max()
        return self

    def transform(self, X):
        return (X - self.min_) / (self.max_ - self.min_)

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X)


#### 5.2 Incorporando MinMax al pipeline [5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [1 punto]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [1 punto]
- Reportar los cambios observados en la distribución de las variables.  [2 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

### 6. Pregunta teórica [5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

Gracias a la experiencia hemos aprendido aque un pipeline permite estandarizar procesos de tratamientos de datos, ya sea eliminar missing values o outliers, esto de manera muy eficiente y facil de manejar. Esto es un atributo y herramienta muy imporante para modelos de machine learning robustos donde se prioriza la optimizacion de procesos.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>